In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')
import os
from utils import *
from student_utils_sp18 import *
from networkx.algorithms.traversal import *
from networkx.algorithms.approximation import *
from vasilis_solver import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
indirectory = "inputs/"
input_files = os.listdir(indirectory)

In [3]:
def draw_graph(G):
    pos = nx.spring_layout(G)
    labels = nx.get_edge_attributes(G,'weight')
    plt.figure(3,figsize=(8,8)) 
    nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
    nx.draw(G, pos, with_labels=True)

In [4]:
def get_G(number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix):
    
    name2index = {}
    index2name = {}

    count = 0
    for name in list_of_kingdom_names:
        name2index[name] = count
        index2name[count] = name
        count += 1

    G = adjacency_matrix_to_graph(adjacency_matrix)
    
    color = {}

    for node in G.nodes():
            color[node] = "white"


    conquering_cost = {}
    for node in G.nodes():
        conquering_cost[node] = adjacency_matrix[node][node]

    nx.set_node_attributes(G, conquering_cost, "conquering_cost")
    nx.set_node_attributes(G, color, "color")
    
    return G, name2index[starting_kingdom]

In [11]:
def solve_all(solver):
    indirectory = "inputs/"
    outdirectory = "outputs/"
    
    #os.makedirs(outdirectory)
    
    infiles = os.listdir(indirectory)
    
    solution_data = {}
    
    for input_file in tqdm(infiles):
            input_data = read_file(indirectory+input_file)
            number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = data_parser(input_data)
            
            G, start_index = get_G(number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix)
            try: 
                closed_walk, conquered_kingdoms = solver(G, start_index)
            except KeyError as e:
                print(e)
                print("Error in input file: {}".format(input_file))
            
            
            # get the score of our solution
            score, text = cost_of_solution(G, closed_walk, conquered_kingdoms)
            
            try:
                conquering_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[0]).group(0)
                travelling_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[1]).group(0)
                total_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[2]).group(0)
                
                solution_data[indirectory+input_file] = {"conquering_cost":conquering_cost, "travelling_cost":travelling_cost,
                                                    "total_cost":total_cost}
            except AttributeError:
                print(indirectory+input_file)
                print(text)
       
            closed_walk = [list_of_kingdom_names[i] for i in closed_walk]
            conquered_kingdoms = [list_of_kingdom_names[i] for i in conquered_kingdoms]
            

            output_file = outdirectory + input_file[:-3] +".out"
            write_data_to_file(output_file, closed_walk, ' ')
            write_to_file(output_file, '\n', append=True)
            write_data_to_file(output_file, conquered_kingdoms, ' ', append=True)
            
    return solution_data

In [14]:
import shutil

shutil.rmtree('outputs/')
os.makedirs("outputs/")
solution_data  = solve_all(vasilis_solver)


 24%|██▍       | 173/716 [01:41<05:17,  1.71it/s]

pipes


 44%|████▍     | 318/716 [03:18<04:08,  1.60it/s]

pipes


 69%|██████▉   | 495/716 [05:04<02:15,  1.63it/s]

...

 91%|█████████ | 648/716 [07:19<00:46,  1.47it/s]

pipes


 93%|█████████▎| 667/716 [07:28<00:32,  1.49it/s]

pipes


100%|██████████| 716/716 [08:10<00:00,  1.46it/s]


In [39]:
total_scores = []
conquering_cost = []
travelling_cost = []
filename = []

for k in solution_data:
    total_scores.append(solution_data[k]['total_cost'])
    conquering_cost.append(solution_data[k]['conquering_cost'])
    travelling_cost.append(solution_data[k]['travelling_cost'])
    filename.append(k)

In [42]:
import pandas as pd

results = pd.DataFrame(data = {'filename':filename, 'total_scores':total_scores, 'conquering_cost':conquering_cost, 
                     'travelling_cost':travelling_cost})
results.head()

,conquering_cost,filename,total_scores,travelling_cost
0,24654.0,inputs/234.in,102494.0,77840.0
1,674538.0,inputs/374.in,60398203.44840002,59723665.44840002
2,2600.0,inputs/265.in,4576.0,1976.0
3,295.0,inputs/325.in,1341.0,1046.0
4,5.0,inputs/296.in,21.0,16.0


In [46]:
results.to_csv("results_log/vasilis_greedy.csv", index=False)

In [50]:
G = nx.Graph()